# Part 1: Build CpG Detector

Here we have a simple problem, given a DNA sequence (of N, A, C, G, T), count the number of CpGs in the sequence (consecutive CGs).

We have defined a few helper functions / parameters for performing this task.

We need you to build a LSTM model and train it to complish this task in PyTorch.

A good solution will be a model that can be trained, with high confidence in correctness.

In [15]:
from typing import Sequence
from functools import partial
import random
import torch
import numpy as np
import random

In [16]:
# DO NOT CHANGE HERE
def set_seed(seed=13):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(13)

# Use this for getting x label
def rand_sequence(n_seqs: int, seq_len: int=128) -> Sequence[int]:
    for i in range(n_seqs):
        yield [random.randint(0, 4) for _ in range(seq_len)]

# Use this for getting y label
def count_cpgs(seq: str) -> int:
    cgs = 0
    for i in range(0, len(seq) - 1):
        dimer = seq[i:i+2]
        # note that seq is a string, not a list
        if dimer == "CG":
            cgs += 1
    return cgs

# Alphabet helpers   
alphabet = 'NACGT'
dna2int = { a: i for a, i in zip(alphabet, range(5))}
int2dna = { i: a for a, i in zip(alphabet, range(5))}

intseq_to_dnaseq = partial(map, int2dna.get)
dnaseq_to_intseq = partial(map, dna2int.get)

In [17]:
# we prepared two datasets for training and evaluation
# training data scale we set to 2048
# we test on 512

def prepare_data(num_samples=100):
    # prepared the training and test data
    # you need to call rand_sequence and count_cpgs here to create the dataset
    # step 1
    X_dna_seqs_train = list(rand_sequence(num_samples))
    """
    hint:
        1. You can check X_dna_seqs_train by print, the data is ids which is your training X 
        2. You first convert ids back to DNA sequence
        3. Then you run count_cpgs which will yield CGs counts - this will be the labels (Y)
    """
     #step2
    temp = ["".join(intseq_to_dnaseq(seq)) for seq in X_dna_seqs_train] # use intseq_to_dnaseq here to convert ids back to DNA seqs
    #step3
    y_dna_seqs =[count_cpgs(seq) for seq in temp] # use count_cpgs here to generate labels with temp generated in step2
    
    return X_dna_seqs_train, y_dna_seqs
    
train_x, train_y = prepare_data(2048)
test_x, test_y = prepare_data(512)

In [18]:
# some config
LSTM_HIDDEN = 256
LSTM_LAYER = 2
batch_size = 64
learning_rate = 0.001
epoch_num = 50
input_size = 128
output_size = 1

In [19]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
# Convert data to PyTorch tensors
train_x = torch.tensor(train_x)
train_y = torch.tensor(train_y)
test_x = torch.tensor(test_x)
test_y = torch.tensor(test_y)

# create data loader
train_data_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=batch_size)
test_data_loader = DataLoader(TensorDataset(test_x, test_y), batch_size=batch_size)

In [28]:
class CpGPredictor(torch.nn.Module):
    ''' Simple model that uses a LSTM to count the number of CpGs in a sequence '''
    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(CpGPredictor, self).__init__()
        self.hidden_size = hidden_size# Store the LSTM's hidden size and number of layers
        self.num_layers = num_layers       
        self.embedding = torch.nn.Embedding(5, input_size)# The embedding layer has a size of 5 and a specified input_size
        # (for 5 possible input types, e.g., A, C, G, T, and unknown) 
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)# Define the LSTM layer, which will process the input sequence and return a hidden state       
        self.fc = torch.nn.Linear(hidden_size, output_size)# The output size corresponds to the number of CpGs predicted by the model

    def forward(self, x):
        ''' Forward pass of the model, defining how the input flows through the network '''
        # Pass the input through the embedding layer to get the embedding vectors for the sequence
        x = self.embedding(x)
        # Both states are initialized as zeros. They must have the correct shape (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)   
        out, _ = self.lstm(x, (h0, c0))# Pass the embedded sequence through the LSTM
        out = self.fc(out[:, -1, :])#This outputs a prediction based on the hidden state from the last time step in the sequence
        return out


In [29]:

model = CpGPredictor(input_size, LSTM_HIDDEN, LSTM_LAYER, output_size)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
# Check if the model is on GPU or CPU
device = next(model.parameters()).device
if device.type == 'cuda':
    print("Model is running on GPU.")
else:
    print("Model is running on CPU.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Model is running on GPU.


In [32]:
# training (you can modify the code below)
t_loss = .0
model.train()
for epoch in range(epoch_num):
    for batch_x, batch_y in train_data_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)# moving to GPU
        # TODO: complete training loop
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = loss_fn(outputs, batch_y.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        
        t_loss += loss.item()

    epoch_loss = t_loss / len(train_data_loader)
    print(f"Epoch {epoch+1}/{epoch_num}, Loss: {epoch_loss}")

    t_loss = .0  # Reset total loss for next epoch


Epoch 1/50, Loss: 7.501315355300903
Epoch 2/50, Loss: 4.217598550021648
Epoch 3/50, Loss: 4.203909769654274
Epoch 4/50, Loss: 4.187906458973885
Epoch 5/50, Loss: 4.091752842068672
Epoch 6/50, Loss: 2.7752440199255943
Epoch 7/50, Loss: 1.4571015927940607
Epoch 8/50, Loss: 0.8811340536922216
Epoch 9/50, Loss: 0.6308174831792712
Epoch 10/50, Loss: 0.4672010447829962
Epoch 11/50, Loss: 0.32654493348672986
Epoch 12/50, Loss: 0.2363478196784854
Epoch 13/50, Loss: 0.14805650408379734
Epoch 14/50, Loss: 0.11473007663153112
Epoch 15/50, Loss: 0.07178926421329379
Epoch 16/50, Loss: 0.06645218364428729
Epoch 17/50, Loss: 0.05641122680390254
Epoch 18/50, Loss: 0.036334607924800366
Epoch 19/50, Loss: 0.026319796248571947
Epoch 20/50, Loss: 0.02112695001414977
Epoch 21/50, Loss: 0.019774569693254307
Epoch 22/50, Loss: 0.016009849510737695
Epoch 23/50, Loss: 0.013339323922991753
Epoch 24/50, Loss: 0.01190568134188652
Epoch 25/50, Loss: 0.011811182310339063
Epoch 26/50, Loss: 0.014748158544534817
Epoc

In [33]:
# Evaluation (you can modify the code below)
model.eval()

# Ensure the model is on the same device as your input data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

res_gs = []
res_pred = []

total_loss = 0
with torch.no_grad():
    for batch_x, batch_y in test_data_loader:
        # Move data to the correct device (GPU if available)
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_x)

        # Save ground truth and predictions for further analysis or metrics
        res_gs.append(batch_y.cpu().numpy())
        res_pred.append(outputs.cpu().numpy())

        # Calculate loss
        loss = loss_fn(outputs, batch_y.float().unsqueeze(1))
        total_loss += loss.item()

# Optionally, you can calculate other metrics like accuracy, etc., here

print(f"Test Loss: {total_loss}")


Test Loss: 0.2082410342991352


In [35]:
# Prepare data
N = 100
test_x, test_y = prepare_data(N)

# Convert data to PyTorch tensors and move to GPU 
test_x = torch.tensor(test_x).to(device)
test_y = torch.tensor(test_y).to(device)

# Ensure the model is on the same device as the data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Run predictions for each sequence and compute overall accuracy
correct_predictions = 0
for sequence, y in zip(test_x, test_y):
    # Perform inference
    with torch.no_grad():
        output = model(sequence.unsqueeze(0).to(device))  # Move sequence to GPU

    # Convert output to CpG count
    predicted_cpg_count = output.item()

    # Check if prediction is within a tolerance of 0.5
    if abs(predicted_cpg_count - y.float().item()) <= 0.3:
        correct_predictions += 1

# Compute overall accuracy
accuracy = correct_predictions / N
print("Overall Accuracy:", accuracy)


Overall Accuracy: 0.95


In [36]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
model_loaded = torch.load('model.pth')



C:\Users\yasha\AppData\Local\Temp\ipykernel_41508\2574230745.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_loaded = torch.load('model.pth')


In [44]:
model.load_state_dict(torch.load('model.pth'))

# Set the model to evaluation mode
model.eval()


C:\Users\yasha\AppData\Local\Temp\ipykernel_41508\3989251230.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))


CpGPredictor(
  (embedding): Embedding(5, 128)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

# Part 2: what if the DNA sequences are not the same length

In [ ]:
# hint we will need following imports
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [ ]:
# DO NOT CHANGE HERE
random.seed(13)

# Use this for getting x label
def rand_sequence_var_len(n_seqs: int, lb: int=16, ub: int=128) -> Sequence[int]:
    for i in range(n_seqs):
        seq_len = random.randint(lb, ub)
        yield [random.randint(1, 5) for _ in range(seq_len)]


# Use this for getting y label
def count_cpgs(seq: str) -> int:
    cgs = 0
    for i in range(0, len(seq) - 1):
        dimer = seq[i:i+2]
        # note that seq is a string, not a list
        if dimer == "CG":
            cgs += 1
    return cgs


# Alphabet helpers   
alphabet = 'NACGT'
dna2int = {a: i for a, i in zip(alphabet, range(1, 6))}
int2dna = {i: a for a, i in zip(alphabet, range(1, 6))}
dna2int.update({"pad": 0})
int2dna.update({0: "<pad>"})

intseq_to_dnaseq = partial(map, int2dna.get)
dnaseq_to_intseq = partial(map, dna2int.get)

In [ ]:
# TODO complete the task based on the change
def prepare_data(num_samples=100, min_len=16, max_len=128):
    # TODO prepared the training and test data
    # you need to call rand_sequence and count_cpgs here to create the dataset
    #step 1
    X_dna_seqs_train = list(rand_sequence_var_len(num_samples, min_len, max_len))
    #step 2
    temp = ???
    #step3
    y_dna_seqs = ???
    
    return X_dna_seqs_train, y_dna_seqs
    
    
min_len, max_len = 64, 128
train_x, train_y = prepare_data(2048, min_len, max_len)
test_x, test_y = prepare_data(512, min_len, max_len)

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, lists, labels) -> None:
        self.lists = lists
        self.labels = labels

    def __getitem__(self, index):
        return torch.LongTensor(self.lists[index]), self.labels[index]

    def __len__(self):
        return len(self.lists)

    
# this will be a collate_fn for dataloader to pad sequence  
class PadSequence:
    #TODO

In [ ]:
# TODO complete the rest